In [1]:
import numpy as np
import h5py
from py15sacla import hdfselection

## create test data

In [2]:
image = np.random.rand(200, 300) - 0.5
filenames = ['test{}.h5'.format(i) for i in range(3)]
run = 5
tag = 73

for filename in filenames:
    with h5py.File(filename, 'w') as h5file:
        h5file.create_dataset('some_irrelevant_thing', data=42)
        for _ in range(5):
            h5file.create_dataset('run_{}/detector/tag_{}'.format(run, tag), data=image+tag)
            tag += 1
        h5file.close()
        run += 1


## select some tags

In [3]:
h5sel_all = [hdfselection.HDFSelection(filename, 'r') for filename in filenames]
tags_to_get = ["tag_{}".format(tag) for tag in [73, 75, 80]]

def filter_tags(hdfsel, tags_to_get):
    hdfout = hdfsel.copy()
    hdfout._datanames = [n for n in hdfout._datanames
                if any(d in n for d in tags_to_get)]
    return hdfout


h5sel_tags = [filter_tags(h5sel, tags_to_get) for h5sel in h5sel_all]

images = np.array([h.value for h5sel in h5sel_tags for h in h5sel])

## confirm this worked

In [4]:
print("The mean of each image is {}".format(np.round(images.mean(axis=(1,2)))))

The mean of each image is [ 73.  75.  80.]
